In [ ]:
import nltk
import networkx as nx
nltk.download('wordnet')
nltk.download('popular')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Download

True

In [ ]:
file = open("topics.301-350.trec6","r")

file = open("topics.351-400.trec7","r")

file = open("topics.401-450.trec8","r")

In [ ]:
file = open("topics.301-350.trec6","r")
full=file.readlines()

title = []
tempdesc = []
tempnarr = []
desc = []
narr = []
desc_flag=0
narr_flag=0
for i in range (0,len(full)):
    if (full[i] == '</top>\n'):
        narr.append(tempnarr)
        tempnarr = []
        continue
    if '<num>' in full[i]:
        continue
    full[i]=full[i].replace('<top>','')
    if '<title>' in full[i]:
        desc_flag=0
        narr_flag=0
        full[i]=full[i].replace('<title>','')
        title.append([full[i],"\n"])
    elif '<desc>' in full[i]:
        desc_flag=1
        narr_flag=0
        full[i]=full[i].replace('<desc> Description:','')
        tempdesc.append(full[i])
    elif '<narr>' in full[i]:
        desc.append(tempdesc)
        tempdesc=[]
        desc_flag=0
        narr_flag=1
        full[i]=full[i].replace('<narr> Narrative:', '')
        tempnarr.append(full[i])
    elif desc_flag:
        tempdesc.append(full[i])
    elif narr_flag:
        tempnarr.append(full[i])


In [ ]:
from nltk.corpus import wordnet as wn

st = open("stopword-list.txt","r")
stpw = st.readlines()

# def get_synonyms(word, pos):
#   ' Gets word synonyms for part of speech '
#   for synset in wn.synsets(word, pos=pos_to_wordnet_pos(pos)):
#     for lemma in synset.lemmas():
#         yield lemma.name()

# def pos_to_wordnet_pos(penntag, returnNone=False):
#     ' Mapping from POS tag word wordnet pos tag '
#     morphy_tag = {'NN':wn.NOUN, 'JJ':wn.ADJ,'VB':wn.VERB, 'RB':wn.ADV}
#     try:
#         return morphy_tag[penntag[:2]]
#     except:
#         return None if returnNone else ''

def expand_query(query):
    expanded_query = []
    for word in query.split():
        if word in stpw:
            continue
        synsets = wn.synsets(word)
        for synset in synsets:
            for lemma in synset.lemmas():
                expanded_query.append((lemma.name(), synset.pos()))
    return expanded_query

def filter(passage):
    G = nx.Graph()
    for term1, pos1 in passage:
        for term2, pos2 in passage:
            if term1 != term2 and pos1 == 'n' and pos2 == 'n':  # Ensure both terms are nouns
                similarity = wn.synset(term1 + '.n.01').path_similarity(wn.synset(term2 + '.n.01'))
                if similarity is not None:
                    G.add_edge(term1, term2, weight=similarity)
            elif term1 != term2 and pos1 == 'v' and pos2 == 'v':  # Ensure both terms are verbs
                similarity = wn.synset(term1 + '.v.01').path_similarity(wn.synset(term2 + '.v.01'))
                if similarity is not None:
                    G.add_edge(term1, term2, weight=similarity)
            elif term1 != term2 and pos1 == 'a' and pos2 == 'a':  # Ensure both terms are adjectives
                similarity=wn.synset(term1 + '.a.01').path_similarity(wn.synset(term2 + '.a.01'))
                if similarity is not None:
                    G.add_edge(term1, term2, weight=similarity)
            elif term1 != term2 and pos1 == 'r' and pos2 == 'r':  # Ensure both terms are adverbs
                similarity=wn.synset(term1 + '.r.01').path_similarity(wn.synset(term2 + '.r.01'))
                if similarity is not None:
                    G.add_edge(term1, term2, weight=similarity)

    central_nodes = sorted(nx.degree_centrality(G).items(), key=lambda x: x[1], reverse=True)[:5]
    filtered_terms = [node for node, centrality in central_nodes]

    return filtered_terms

def synonym_add(doc):
  for txt in doc:
    temptxt = []
    text = nltk.word_tokenize(txt)
    for i in text:
      if i+'\n' in stpw:
        continue
      else:
        temptxt.append(i)
    text = temptxt
    alttext = " ".join(str(element) for element in text)


    finaltext = filter(expand_query(alttext))

    finaltext = alttext + " " + " ".join(str(element) for element in finaltext)
    f.write(finaltext +"\n")
    print(finaltext)

In [ ]:
f = open("alt-topics.301-350.trec6","w")
file = open("topics.301-350.trec6","r")

f = open("alt-topics.351-400.trec7","w")
file = open("topics.351-400.trec7","r")

f = open("alt-topics.401-450.trec8","w")
file = open("topics.401-450.trec8","r")

In [ ]:
f = open("alt-topics.301-350.trec6","w")
file = open("topics.301-350.trec6","r")
full2 = file.readlines()
titlef = 0
descf = 0
narrf = 0
desc_flag=0
narr_flag=0
cnt = 0


for p in range(len(full2)):
  if '<title>' in full2[p]:
    f.write('<title> ')
    synonym_add(title[cnt])
  elif '</top>' in full2[p]:
    desc_flag=0
    narr_flag=0
    f.write(full2[p])
    cnt = cnt+1
  elif '<desc>' in full2[p]:
    desc_flag=1
    narr_flag=0
    f.write("<desc> Description:")
    synonym_add(desc[cnt])
  elif '<narr>' in full2[p]:
    desc_flag=0
    narr_flag=1
    f.write("<narr> Narrative:")
    synonym_add(narr[cnt])
  elif desc_flag == 0 and narr_flag == 0:
    f.write(full2[p])



f.close
file.close


foreign minorities , Germany minority nonage Germany Federal_Republic_of_Germany Deutschland
 
 
What language cultural differences impede integration language linguistic_communication speech speech_communication spoken_communication
foreign minorities Germany ? minority nonage Germany Federal_Republic_of_Germany Deutschland
 
 
A relevant document will focus causes lack document will focus cause lack
integration significant way ; , mere mention mention cite way integration integrating
immigration difficulties relevant . Documents discuss document immigration in-migration trouble difficulty
immigration problems unrelated Germany relevant . immigration in-migration problem job trouble
 
behavioral genetics behavioral behavioural genetics genetic_science
 
 
What happening field behavioral genetics , field happening occurrence occurrent natural_event
study relative influence genetic study work influence relative analyze
environmental factors individual 's behavior individual factor compo

<function TextIOWrapper.close()>